In [1]:
from pfp_stat import *

import numpy as np
import pandas as pd 

import pandas_datareader.data as web
import matplotlib.pyplot as plt
import matplotlib

import datetime as dat

from pylab import title, figure, xlabel, ylabel, xticks, bar, legend, axis, savefig
from fpdf import FPDF
import fpdf

from pfp_products import *

import os.path
from scipy.optimize import minimize

In [2]:
def mirr(wghts):
    a1 = (payoffs*wghts).sum(axis = 2)    
    a2 = np.ones((a1.shape[0]+1,a1.shape[1]))*(-1)
    a2[1:,:] = a1
    a3 = np.array([np.irr(x) for x in np.transpose(a2)]) 
    return a3

In [3]:
print('Okay, lets see what we have here.')

prodcat_filename = 'inputs/prod_cat open.xlsx'
prodcat_sheetname = 'prod cat'

xls = pd.read_excel(prodcat_filename, prodcat_sheetname, 
                         decimal = '.', usecols = 'b:z', 
                         index_col = 0, header = 1)

print('I`m using `' + prodcat_sheetname + '` sheet of `' + prodcat_filename + '` file as product catalog.')

products = xls.transpose()
all_BAs = []
prod_list = []
max_term = 0

for prod, row in products.iterrows():
    
    check = check_stat(row.BAs)
    
    if check:        
        all_BAs.extend(row.BAs.split(', '))
        all_BAs = list(set(all_BAs))
        
        if row.term > max_term: max_term = row.term

        Note1 = Structure(prod, row.BAs, row.notional_curr, row.term, 
                  row.coupon_value, row.coupon_always, row.coupon_check_months, row.coupon_memory, 
                  row.coupon_lower_barrier, row.coupon_upper_barrier,
                  row.autocall_flag, row.autocall_check_months, row.autocall_barrier, row.autocall_barrier_increase_rate,
                  row.redemption_amount, row.redemption_put_strike, row.redemption_guarantee_rule,
                  row.redemption_upside_participation, row.redemption_downside_participation, row.issuer)
        Note1.stats_ok = True
        
        prod_list.append(Note1)
    else: 
        print ('We dont have enough statistics for ' + row.name + ' underlyings. We do not calculate it now.')

print('...')
print('All products from `prod cat.xls` except mentioned above are successfully loaded.')

Okay, lets see what we have here.
I`m using `prod cat` sheet of `inputs/prod_cat open.xlsx` file as product catalog.
...
All products from `prod cat.xls` except mentioned above are successfully loaded.


In [4]:
n_scenarios = 50000
simulation_years = max_term   

returns = ba_scenarios(all_BAs, 
                       simulation_years,  
                       n_scenarios, 
                       print_statistics = True,
                       points_in_year = 12)
print('Scenarios ready!')

Returns: 
gmkn rx equity    0.079
mtss rx equity    0.092
moex rx equity    0.082
open pif          0.121
solecmvt index    0.025
yndx rx equity    0.121
alrs rx equity    0.087
rog sw equity     0.034
Name: r, dtype: float64
 
Sigmas: 
gmkn rx equity    0.191840
mtss rx equity    0.249655
moex rx equity    0.213164
open pif          0.143242
solecmvt index    0.096891
yndx rx equity    0.323618
alrs rx equity    0.278227
rog sw equity     0.156097
dtype: float32
 
Correlations: 

                gmkn rx equity  mtss rx equity  moex rx equity  open pif  \
gmkn rx equity        1.000000        0.342554        0.276289  0.406332   
mtss rx equity        0.342554        1.000000        0.299780  0.414267   
moex rx equity        0.276289        0.299780        1.000000  0.670502   
open pif              0.406332        0.414267        0.670502  1.000000   
solecmvt index        0.291811        0.274833        0.302035  0.074107   
yndx rx equity        0.013790        0.170849        0.52

In [5]:
flag = 1
for prod in prod_list:
    
    print('calculating ' + prod.name)
    
    a1 = prod.payoff(all_BAs, returns, to_pdf = False)
    # a1 - массив размера self.time_steps x n_scenarios
    
    a2 = np.ones((a1.shape[0]+1,a1.shape[1]))*(-1)
    a2[1:,:] = a1
    a3 = np.array([np.irr(x) for x in np.transpose(a2)])
    
    x = a3.std()
    y = a3.mean()
    z = a3.min()

    if flag == 1:
        flag = 0
        payoffs = a1
        irrs = a3
        points1 = [x, y]
        points2 = [z, y]
        
        names = [prod.name]
    else:
        payoffs = np.dstack([payoffs, a1])
        irrs = np.c_[irrs, a3]
        points1 = np.c_[points1, [x, y]]
        points2 = np.c_[points2, [z, y]]
        names.append(prod.name)
print('Done!')

calculating GMKN AM
calculating Rosche AM
calculating Basket coupon
calculating Open PIF
calculating Solars
Done!


In [6]:
payoffs.shape

(3, 50000, 5)

In [7]:
payoffs

array([[[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.11      , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.11      , 0.        , 0.        ]],

       [[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.22      , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.11      , 0.        , 0.        ]],

       [[1.39411568, 1.03531529, 1.        , 1.81839839, 1.02201724],
        [1.        , 1.        , 1.        , 1.62563769, 1. 